In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform==1.60.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 57.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-00-6f7e844db878


In [2]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [7]:
# Import the required libraries
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

# TODO: Define a function to reverse the order
# of a string and return the result.
# Keep the print statement within the function.

def reverse_string(s):
    print("Calling reverse function")
    return s[::-1]

# TODO: Define a function to remove white space
# characters from a string and return the result.
# Keep the print statement within the function.

def remove_white_spaces(s):
    print("Calling remove_white_spaces function")
    return s.replace(' ', '')

# TODO: Create FunctionDeclarations for your functions

reverse_string_function_declaration = FunctionDeclaration(
    name="reverse_string",
    description="Reverses the order of a string.",
    parameters={
        "type": "object",
        "properties": {
            "s": {
                "type": "string",
                "description": "The string to reverse."
            }
        },
        "required": ["s"]
    }
)

remove_white_spaces_function_declaration = FunctionDeclaration(
    name="remove_white_spaces",
    description="Removes space characters from a string.",
    parameters={
        "type": "object",
        "properties": {
            "s": {
                "type": "string",
                "description": "The string to process."
            }
        },
        "required": ["s"]
    }
)


In [9]:
string_tool = Tool(
    function_declarations=[
        reverse_string_function_declaration,
        remove_white_spaces_function_declaration
    ]
)

In [10]:
model = GenerativeModel(
    model_name="gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[string_tool],
    system_instruction=[
        "Fulfill the user's instructions.",
        "If asked to reverse a string or remove whitespace, call the provided functions.",
        "You may call one function after the other if needed."
        "Repeat the result to the user.",
    ],
)

In [11]:
chat = model.start_chat()

In [12]:
response = chat.send_message("reverse the string 'ARSENAL' ")

In [13]:
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "reverse_string"
        args {
          fields {
            key: "s"
            value {
              string_value: "ARSENAL"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.1943359375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1591796875
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: LOW
    probability_score: 0.5390625
    severity: HARM_SEVERITY_LOW
    severity_score: 0.392578125
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.1767578125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1259765625
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.1

In [15]:
def handle_response(response):
    # If there is a function call then invoke it
    # Otherwise print the response.
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        print(response.text)
        return
    
    # Extract the function name
    function_name = function_call.name
    
    if function_name == "reverse_string":
        # Extract the arguments to use in your function
        arguments = function_call.args
        import json
        args = arguments
        s = args.get("s")
        # Call your function
        result = reverse_string(s)
        # Send the result back to the chat session with the model
        new_response = chat.send_message(str(result))
        # Recursive call
        handle_response(new_response)
    
    elif function_name == "remove_white_spaces":
        # Extract the arguments to use in your function
        arguments = function_call.args
        import json
        args = arguments
        s = args.get("s")
        # Call your function
        result = remove_white_spaces(s)
        # Send the result back to the chat session with the model
        new_response = chat.send_message(str(result))
        # Make a recursive call of this handler function
        handle_response(new_response)
    
    else:
        # You shouldn't end up here
        print(function_call)


In [16]:
response = chat.send_message("Tell me a joke.")
handle_response(response)

Why don't scientists trust atoms?

Because they make up everything! 



In [17]:
response = chat.send_message("Reverse the following string: Hello world")
handle_response(response)

Calling reverse function
dlrow olleH 



In [18]:
response = chat.send_message("Reverse the following string: LANESRA")
handle_response(response)

Calling reverse function
ARSENAL 



In [19]:
response = chat.send_message("Remove the white spaces from the following word: hello @ world. com")
handle_response(response)

Calling remove_white_spaces function
hello@world.com 



In [20]:
response = chat.send_message("Reverse the following string and remove spaces: I do not understand the new sentence.")
handle_response(response)   

Calling reverse function
Calling remove_white_spaces function
.ecnetneswenehtdnatsrednutonodI 



In [21]:
response = chat.send_message("Tell me a fact. Then remove the spaces in the fact text and then reverse the fact text.")
print(response.candidates[0].content.parts[0].text)
handle_response(response)

A group of owls is called a parliament. 


Calling remove_white_spaces function
Calling reverse function
.tnemailrapadellacsislwofopuorgA 

